In [1]:
import numpy as np

In [2]:
def iter_query_cog(query_orth):
    with open(query_orth) as csvfile:
        next(csvfile)
        for l in csvfile:
            buffer = l.split(';')
            yield([buffer[4],buffer[5]])

In [3]:
def cog_2_g_list(cog_id,cog_dir):
    file_path = f'{cog_dir}/COG_{cog_id}.csv'
    g_list = list()
    with open(file_path) as csvfile:
        next(csvfile)
        for l in csvfile:
            buffer = l.split(',')
            strain = buffer[1]
            g_list.append(strain)
    return g_list

In [53]:
def p_dict_builder(query_orth,cog_dir):
    p_dict={}
    i=0
    for cog_id,p_name in iter_query_cog(query_orth):
        g_list = cog_2_g_list(cog_id,cog_dir)
        p_dict[p_name] = g_list
        #i=i+1
        #if i>9:
            #break
    return p_dict

In [38]:
def index_setter(p_dict):
    p_index = list()
    g_index = set()
    for p_name,g_list in p_dict.items():
        p_index.append(p_name)
        g_index.update(g_list)
    g_index = list(g_index)
    return p_index,g_index

In [49]:
def set_M_values(g_name,p_name,value): 
    i = m_index[1].index(g_name) #genome index
    j = m_index[0].index(p_name) #protein index
    M[i,j] = value

In [54]:
%%time
#query_orth ='/Users/mdupuy/Documents/Stage/Pseudomonas_aeruginosa_PA7_119_ortholog_groups.csv'
#cog_dir='/Users/mdupuy/Documents/Stage/All_COG_groups/'
p_dict=p_dict_builder(query_orth,cog_dir)
m_index=index_setter(p_dict)
x = len(m_index[1])
y = len(m_index[0])
M = np.zeros((x,y))
for p_name,g_list in p_dict.items():
    for g_name in g_list:
        set_M_values(g_name,p_name,1)

CPU times: user 4min 25s, sys: 5.87 s, total: 4min 31s
Wall time: 5min 58s
